## Convert static computational graph to ONNX

#### Prepare environment

In [ ]:
!pip install paddlepaddle paddle2onnx # required, paddlepaddle>=1.8.0, paddle2onnx>=0.3.2
!pip install onnx onnxruntime # optional for check and run ONNX model, onnx>=1.7.0, onnxruntime>=1.5.2

In [1]:
import paddle.fluid as fluid
import os

### 1. ResNet network

In [2]:
def conv_bn_layer(input,
                  ch_out,
                  filter_size,
                  stride,
                  padding,
                  act='relu',
                  bias_attr=False):
    tmp = fluid.layers.conv2d(
        input=input,
        filter_size=filter_size,
        num_filters=ch_out,
        stride=stride,
        padding=padding,
        act=None,
        bias_attr=bias_attr)
    return fluid.layers.batch_norm(input=tmp, act=act)

def shortcut(input, ch_in, ch_out, stride):
    if ch_in != ch_out:
        return conv_bn_layer(input, ch_out, 1, stride, 0, None)
    else:
        return input

def basicblock(input, ch_in, ch_out, stride):
    tmp = conv_bn_layer(input, ch_out, 3, stride, 1)
    tmp = conv_bn_layer(tmp, ch_out, 3, 1, 1, act=None, bias_attr=True)
    short = shortcut(input, ch_in, ch_out, stride)
    return fluid.layers.elementwise_add(x=tmp, y=short, act='relu')

def layer_warp(block_func, input, ch_in, ch_out, count, stride):
    tmp = block_func(input, ch_in, ch_out, stride)
    for i in range(1, count):
        tmp = block_func(tmp, ch_out, ch_out, 1)
    return tmp

def resnet_cifar10(ipt, depth=32):
    # depth should be one of 20, 32, 44, 56, 110, 1202
    assert (depth - 2) % 6 == 0
    n = (depth - 2) // 6
    nStages = {16, 64, 128}
    conv1 = conv_bn_layer(ipt, ch_out=16, filter_size=3, stride=1, padding=1)
    res1 = layer_warp(basicblock, conv1, 16, 16, n, 1)
    #res2 = layer_warp(basicblock, res1, 16, 32, n, 2)
    #res3 = layer_warp(basicblock, res2, 32, 64, n, 2)
    pool = fluid.layers.pool2d(
        input=res1, pool_size=8, pool_type='avg', pool_stride=1)
    predict = fluid.layers.fc(input=pool, size=10, act='softmax')
    return predict

data_shape = [None, 3, 32, 32]
images = fluid.data(name='pixel', shape=data_shape, dtype='float32')
predict = resnet_cifar10(images, 32)
exe = fluid.Executor(fluid.CPUPlace())
_ =exe.run(fluid.default_startup_program())

### 2. Convert uncombined PaddlePaddle model(parameters saved in separated files)

In [3]:
model_dir = './resnet_not_combined/'
fluid.io.save_inference_model(model_dir, ["pixel"], [predict], exe)

['save_infer_model/scale_0.tmp_0']

In [4]:
!ls ./resnet_not_combined/

__model__         batch_norm_2.w_2  batch_norm_6.w_2  conv2d_10.w_0
batch_norm_0.b_0  batch_norm_3.b_0  batch_norm_7.b_0  conv2d_2.b_0
batch_norm_0.w_0  batch_norm_3.w_0  batch_norm_7.w_0  conv2d_2.w_0
batch_norm_0.w_1  batch_norm_3.w_1  batch_norm_7.w_1  conv2d_3.w_0
batch_norm_0.w_2  batch_norm_3.w_2  batch_norm_7.w_2  conv2d_4.b_0
batch_norm_1.b_0  batch_norm_4.b_0  batch_norm_8.b_0  conv2d_4.w_0
batch_norm_1.w_0  batch_norm_4.w_0  batch_norm_8.w_0  conv2d_5.w_0
batch_norm_1.w_1  batch_norm_4.w_1  batch_norm_8.w_1  conv2d_6.b_0
batch_norm_1.w_2  batch_norm_4.w_2  batch_norm_8.w_2  conv2d_6.w_0
batch_norm_10.b_0 batch_norm_5.b_0  batch_norm_9.b_0  conv2d_7.w_0
batch_norm_10.w_0 batch_norm_5.w_0  batch_norm_9.w_0  conv2d_8.b_0
batch_norm_10.w_1 batch_norm_5.w_1  batch_norm_9.w_1  conv2d_8.w_0
batch_norm_10.w_2 batch_norm_5.w_2  batch_norm_9.w_2  conv2d_9.w_0
batch_norm_2.b_0  batch_norm_6.b_0  conv2d_0.w_0      fc_0.b_0
batch_norm_2.w_0  batch_norm_6.w_0  conv2d_1.w_0      fc_0.w_0
ba

In [ ]:
# Execute the shell command. Jupyter may can't find the paddle2onnx command, 
# please go to the command line to execute

!paddle2onnx --model_dir ./resnet_not_combined/  --save_file onnx-model/model.onnx --opset_version 10

## 3. Convert combined PaddlePaddle model(parameters saved in one binary file)

In [5]:
model_dir = './resnet_combined/'
fluid.io.save_inference_model(model_dir, ["pixel"], [predict], exe, model_filename='__model__', params_filename='__params__')

['save_infer_model/scale_0.tmp_1']

In [6]:
!ls ./resnet_combined/

__model__  __params__


In [ ]:
# Execute the shell command. Jupyter may can't find the paddle2onnx command, 
# please go to the command line to execute

!paddle2onnx --model_dir ./resnet_combined/ --model_filename '__model__' --params_filename '__params__' --save_file onnx-model/model.onnx --opset_version 10